In [ ]:
# !pip install -U iminizinc

In [ ]:
# Import the necessary libraries
from tqdm import tqdm
from utils import import_instances, plot_solution, output_solution
import time
%reload_ext iminizinc

## Data Import

In [ ]:
# Get all instances from text files
instances = import_instances('input/instances/')

## Constraint Programming

In [ ]:
def get_variables(NUMBER):
    # Get the number of blocks
    BLOCKS = []
    for n in range(int(instances[NUMBER][1])):
        BLOCKS.append(f'Block {n}')
    
    # Get block lengths and heights
    BLOCK_WIDTHS = []
    BLOCK_HEIGHTS = []

    for value in instances[NUMBER][2:]:
        width, height = value.split(' ')
        BLOCK_WIDTHS.append(int(width))
        BLOCK_HEIGHTS.append(int(height))
    
    # Get the maximum width and height
    MAX_WIDTH = int(instances[NUMBER][0])
    MAX_HEIGHT = sum(BLOCK_HEIGHTS)
    
    return BLOCKS, BLOCK_WIDTHS, BLOCK_HEIGHTS, MAX_WIDTH, MAX_HEIGHT

In [ ]:
%%mzn_model model

include "globals.mzn";

enum BLOCKS;
array[BLOCKS] of int: BLOCK_WIDTHS;
array[BLOCKS] of int: BLOCK_HEIGHTS;

int: MAX_WIDTH;
int: MAX_HEIGHT;

% Constraints to find y-coordinates
array[BLOCKS] of var 0..MAX_HEIGHT: start_y;
var 0..MAX_HEIGHT: end_y;

constraint cumulative(start_y, BLOCK_HEIGHTS, BLOCK_WIDTHS, MAX_WIDTH);

constraint forall(b in BLOCKS)(start_y[b] + BLOCK_HEIGHTS[b] <= end_y);

% Constraints to find x-coordinates
array[BLOCKS] of var 0..MAX_WIDTH: start_x;

constraint cumulative(start_x, BLOCK_WIDTHS, BLOCK_HEIGHTS, end_y);

constraint forall(b in BLOCKS)(start_x[b] + BLOCK_WIDTHS[b] <= MAX_WIDTH);

solve minimize end_y;

In [ ]:
# Show the solution found
def show_solution(start_x, start_y, end_y, file=None):    
    circuits = []
    for i in range(len(BLOCKS)):
        circuits.append([BLOCK_WIDTHS[i], BLOCK_HEIGHTS[i], start_x[i], start_y[i]])

    plot_solution(MAX_WIDTH, end_y, len(BLOCKS), circuits, file)

In [ ]:
%%mzn_model model

include "globals.mzn";

enum BLOCKS;
array[BLOCKS] of int: BLOCK_WIDTHS;
array[BLOCKS] of int: BLOCK_HEIGHTS;

int: MAX_WIDTH;
int: MAX_HEIGHT;

% Constraints to find y-coordinates
array[BLOCKS] of var 0..MAX_HEIGHT: start_y;
var 0..MAX_HEIGHT: end_y;

constraint cumulative(start_y, BLOCK_HEIGHTS, BLOCK_WIDTHS, MAX_WIDTH);

constraint forall(b in BLOCKS)(start_y[b] + BLOCK_HEIGHTS[b] <= end_y);

% Constraints to find x-coordinates
array[BLOCKS] of var 0..MAX_WIDTH: start_x;

constraint cumulative(start_x, BLOCK_WIDTHS, BLOCK_HEIGHTS, end_y);

constraint forall(b in BLOCKS)(start_x[b] + BLOCK_WIDTHS[b] <= MAX_WIDTH);

solve minimize end_y;

In [ ]:
# Show a sample solution
NUMBER = 0
BLOCKS, BLOCK_WIDTHS, BLOCK_HEIGHTS, MAX_WIDTH, MAX_HEIGHT = get_variables(NUMBER)

timeout = time.time() + 60*5
%minizinc model --time-limit 300000 -m bind

if time.time() >= timeout:
    print(f'Instance-{NUMBER+1} Fail: Timeout')
else:
    show_solution(start_x, start_y, end_y)

## Data Output

In [ ]:
for n in tqdm(range(len(instances))):
    NUMBER = n
    BLOCKS, BLOCK_WIDTHS, BLOCK_HEIGHTS, MAX_WIDTH, MAX_HEIGHT = get_variables(NUMBER)
    
    timeout = time.time() + 60*5
    %minizinc model --time-limit 300000 -m bind
    
    if time.time() >= timeout:
        print(f'Instance-{NUMBER+1} Fail: Timeout')
    else:
        output_solution(instances[NUMBER], start_x, start_y, end_y, f'output/CP/solutions/out-{NUMBER+1}.txt')
        show_solution(start_x, start_y, end_y, f'output/CP/images/out-{NUMBER+1}.png')